Import the required libraries and the combined dataset mike b created

Drop all rows where any of the data elements are NA

In [9]:
import geopandas as gpd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

Pre-processing of the datasets
- Remove areas with max veg heights > 0.5 m

In [10]:
df = gpd.read_file('./data/shapefiles/geodata.shp')
df = siksik.dropna()

veg_max = 0.5 # Max veg height we're willing to deal with
df = df[df['vegheight_']<= veg_max]

In [14]:
snow_max = np.round(df.snow_depth.max()+0.5)
snow_min = 0.1
snow_step = 0.01 #Bin width of 1 cm
bsnow = np.arange(snow_min,snow_max,snow_step) #These are the histogram bins

Here we calculate a probability density function (PDF), invert its cumulative density (CDF).
The bins are bound by the observations so we don't extrapolate.

In [15]:
hist, bin_edges = np.histogram(siksik_reduced.snow_depth, bins=bsnow, density=True)
cum_values = np.zeros(bin_edges.shape)
cum_values[1:] = np.cumsum(hist*np.diff(bin_edges))
inv_cdf = interpolate.interp1d(cum_values, bin_edges)

Example of how to pull random samples from the distrobution
1. Pick number of samples
2. Generate random numbers between 0-1
3. Use the inverted cdf to pull samples from the known distro
4. Find nearest match in the obs. Need to do this because we interpolate the cdf.

If you pull enough samples, you'll recreate the observed distrobution.

In [87]:
n_samples = 50
r = np.random.rand(n_samples)
depth_sample = inv_cdf(r)
sample_records = [df.iloc[(df['snow_depth']-x).abs().argsort()[:1]] for x in depth_sample]

print(depth_sample[0])
print(sample_records[0].snow_depth)

0.5696214825184002
13886    0.56964
Name: snow_depth, dtype: float64


In [100]:
print('2013 Lidar Snow Stats for SikSik')
print ("Mean: {} ".format(np.round(df.snow_depth.mean(),2)))
print ("StDev: {} ".format(np.round(df.snow_depth.std(),2)))

print('\n2018 Site Stats')
print ("Mean: {} ".format(np.round(depth_sample.mean(),2)))
print ("StDev: {} ".format(np.round(depth_sample.std(),2)))

2013 Lidar Snow Stats for SikSik
Mean: 0.54 
StDev: 0.12 

2018 Site Stats
Mean: 0.54 
StDev: 0.1 
